This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [ ]:
from pydrake.all import ModelVisualizer, PackageMap, Simulator, StartMeshcat

from manipulation import ConfigureParser, FindResource, running_as_notebook
from manipulation.remotes import AddSpotRemote
from manipulation.station import LoadScenario, MakeHardwareStation

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Simplified Spot model for mobile manipulation

First we'll use the ModelVisualizer to inspect the model.

In [ ]:
visualizer = ModelVisualizer(meshcat=meshcat)
ConfigureParser(visualizer.parser())
AddSpotRemote(visualizer.parser().package_map())
visualizer.AddModels(
    url="package://manipulation/spot/spot_with_arm_and_floating_base_actuators.urdf"
)
visualizer.Run(loop_once=not running_as_notebook)
meshcat.DeleteAddedControls()

Now we can use HardwareStation to create a basic simulation.

In [ ]:
scenario = LoadScenario(
    filename=FindResource(
        "models/spot/spot_with_arm_and_floating_base_actuators.scenario.yaml"
    )
)
station = MakeHardwareStation(
    scenario,
    meshcat,
    parser_preload_callback=lambda parser: AddSpotRemote(parser.package_map()),
)
simulator = Simulator(station)
context = simulator.get_mutable_context()
x0 = station.GetOutputPort("spot.state_estimated").Eval(context)
station.GetInputPort("spot.desired_state").FixValue(context, x0)
simulator.AdvanceTo(0.1);